### Importando Bibliotecas

In [29]:
import urllib, json
import pandas as pd
import urllib, json
from tqdm.notebook import tqdm
from datetime import datetime
import psycopg2


### Informações da API

In [30]:
url = 'https://monitoramento.semas.pa.gov.br/monitoramento/AgendaVerde/ceprofsAtivos'
with urllib.request.urlopen(url) as url:
    data = json.loads(url.read().decode())

df = pd.json_normalize(data)
df_pmfs = df[df['classificacao'] == 'PMFS']
df_pmfs.columns = ['id', 'cpf_cnpj', 'classificacao', 'nome', 'y', 'x', 'municipio_id', 'municipio_nome']

### Banco de Dados

### Criando Banco de Dados

In [31]:
import psycopg2

conn = psycopg2.connect(
   database="postgres",
    user='postgres',
    password='postgres',
    host='localhost',
    port= '5432'
)
  
conn.autocommit = True
  
# Creating a cursor object
cursor = conn.cursor()
# query to create a database
sql = 'CREATE database pmfs'
try:
    cursor.execute(sql)
    conn.close()
except Exception as e:
    print(e)
    pass

database "pmfs" already exists



### Criando Tabelas

#### Tabela PMFS

In [32]:
conn = psycopg2.connect(
   database="pmfs",
    user='postgres',
    password='postgres',
    host='localhost',
    port= '5432'
)
  
conn.autocommit = True
  
# Creating a cursor object
cursor = conn.cursor()
# query to create a database 
sql = """
    CREATE TABLE IF NOT EXISTS public.pmfs
(
    id bigint NOT NULL,
    cpf_cnpj character varying(20) COLLATE pg_catalog."default" NOT NULL,
    classificacao character varying(5) COLLATE pg_catalog."default" DEFAULT 'pmfs'::character varying,
    nome character varying(250) COLLATE pg_catalog."default" NOT NULL,
    x double precision NOT NULL,
    y double precision NOT NULL,
    municipio_id bigint NOT NULL,
    municipio_nome character varying(150) COLLATE pg_catalog."default" NOT NULL,
    CONSTRAINT pmfs_pkey PRIMARY KEY (id)
);
"""
cursor.execute(sql)
conn.close()

#### Tabela Detalhamento

In [33]:
conn = psycopg2.connect(
   database="pmfs",
    user='postgres',
    password='postgres',
    host='localhost',
    port= '5432'
)
  
conn.autocommit = True
  
# Creating a cursor object
cursor = conn.cursor()
# query to create a database 
sql = """
CREATE TABLE IF NOT EXISTS public.detalhamento_pmfs
(
    id bigint NOT NULL GENERATED ALWAYS AS IDENTITY ( INCREMENT 1 START 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 ),
    numero_autef integer NOT NULL,
    data_vencimento date NOT NULL,
    nome_produto character varying(50) COLLATE pg_catalog."default" NOT NULL,
    unidade_produto character varying(10) COLLATE pg_catalog."default" NOT NULL,
    volume_autorizado double precision NOT NULL,
    volume_debitado double precision NOT NULL,
    area_autorizada double precision NOT NULL,
    data_insercao date NOT NULL,
    id_pmfs bigint NOT NULL,
    saldo double precision,
    numero_protocolo character varying(50) COLLATE pg_catalog."default" NOT NULL,
    CONSTRAINT detalhamento_pmfs_pkey PRIMARY KEY (id),
    CONSTRAINT "PMFS_DETALHAMENTO_FK" FOREIGN KEY (id_pmfs)
        REFERENCES public.pmfs (id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
        NOT VALID
);

"""

cursor.execute(sql)
conn.close()

### Inserções

#### Inserções no Banco pmfs

In [44]:
','.join(df_detalhamentos.columns.tolist())

'id_pmfs,numero_autef,area_autorizada,numero_protocolo,data_vencimento,nome_produto,unidade_produto,volume_autorizado,volume_debitado,saldo,data_insercao'

In [52]:
conn = psycopg2.connect(
   database="pmfs",
    user='postgres',
    password='postgres',
    host='localhost',
    port= '5432'
)

conn.autocommit = True
cursor = conn.cursor()
cols = ",".join([str(i) for i in df_pmfs.columns.tolist()])


for i,row in df_pmfs.iterrows():
    sql = "INSERT INTO public.pmfs (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s) ON CONFLICT DO NOTHING"
    cursor.execute(sql, tuple(row))

conn.commit()
print('Inserções Concluídas com Sucesso!')

Inserções Concluídas com Sucesso!


#### Requisitando detalhamento no PMFS

In [35]:
# areaAutorizada numeroProtocolo numero dataVencimento nomeProduto unidadeProduto volumeAutorizado volumeDebitado
ids = df_pmfs['id'].tolist()
detalhamentos_json = []

for id in tqdm(range(0,len(ids)), desc='Inserções de detalhamentos pmfs'):
    nome_pmfs = ids[id]
    URL_BASE = 'https://monitoramento.semas.pa.gov.br/monitoramento/AgendaVerde/dadosAutefByCeprof?codigoCeprof=' + str(ids[id])
    # url = 'https://monitoramento.semas.pa.gov.br/monitoramento/AgendaVerde/ceprofsAtivos'

    with urllib.request.urlopen(URL_BASE) as url:
        detalhamento_data = json.loads(url.read().decode())

    for data in detalhamento_data:
        try:
            infos = {'id_pmfs': data['ceprof']['id'], 'numero_autef': data['numero'],
                     'area_autorizada': data['areaAutorizada'], 'numero_protocolo': data['numeroProtocolo'],
                     'data_vencimento': data['dataVencimento'], 'nome_produto':data['detalhamentos'][0]['nomeProduto'],
                     'unidade_produto': data['detalhamentos'][0]['unidadeProduto'], 'volume_autorizado': data['detalhamentos'][0]['volumeAutorizado'],
                     'volume_debitado': data['detalhamentos'][0]['volumeDebitado'], 'saldo': data['detalhamentos'][0]['volumeAutorizado'] - data['detalhamentos'][0]['volumeDebitado'] ,'data_insercao': datetime.now().strftime("%Y-%m-%d %H:%M")
                    }
        except Exception as e:
            #print('error:', e)
            #print('id:', id)
            pass
        
        detalhamentos_json.append(infos)
df_detalhamentos = pd.json_normalize(detalhamentos_json)


Inserções de detalhamentos pmfs:   0%|          | 0/372 [00:00<?, ?it/s]

In [51]:
# df_detalhamentos

#### Inserções detalhamentos PMFS

In [50]:
import psycopg2
conn = psycopg2.connect(
   database="pmfs",
    user='postgres',
    password='postgres',
    host='localhost',
    port= '5432'
)

conn.autocommit = True
cursor = conn.cursor()
cols = ",".join([str(i) for i in df_detalhamentos.columns.tolist()])

for i,row in df_detalhamentos.iterrows():
    sql = "INSERT INTO public.detalhamento_pmfs (" +cols + ") VALUES (" + "%s,"*(len(row)-1) + "%s) ON CONFLICT ON CONSTRAINT pmfs_detalhamento_uk DO NOTHING"
    cursor.execute(sql, tuple(row))

conn.commit()
print('Inserções Efetuadas com Sucesso')

Inserções Efetuadas com Sucesso


In [38]:
# for data in detalhamento_data:
# #     areaAutorizada numeroProtocolo numero dataVencimento nomeProduto unidadeProduto volumeAutorizado volumeDebitado
#     print(data['ceprof']['id'])
#     print(data['numero'], data['areaAutorizada'], data['numeroProtocolo'], data['dataVencimento'])
#     print(data['detalhamentos'][0]['nomeProduto'], data['detalhamentos'][0]['unidadeProduto'], data['detalhamentos'][0]['volumeAutorizado'], data['detalhamentos'][0]['volumeDebitado'])